<a href="https://colab.research.google.com/github/Cassini-chris/Transfer-Learning-Image-Classification/blob/master/Transfer_Learning_Kardashians_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import packages

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

import os
import zipfile

In [ ]:
#Check current directory
!pwd
#Go to directory
os.chdir('/tmp')
!pwd

#Remove Folder
#!rm -rf Kardashians4

## Load data

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

#--- Input --- Location of ZIP File
LOCATION_ZIP = '/content/gdrive/My Drive/__TECH/_My Flask Apps/Disney Picture Material/disney_princesses_dataset.zip'
FOLDER_NAME = 'Kardashians'

#---Create Directory--- to store the training / validation data
try:
  os.mkdir (FOLDER_NAME)
  os.chdir(FOLDER_NAME)
except OSError:
    pass

#Unzip our .zip file in the directory
#!unzip "/content/gdrive/My Drive/__TECH/_My Flask Apps/Disney Picture Material/disney_princesses_dataset.zip" -d 'Kardashians'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/'+FOLDER_NAME)
zip_ref.close()

#Declare path__
PATH = '/tmp/' + FOLDER_NAME

In [ ]:
#Define image path of our Prediction Image
image_path = "/content/gdrive/My Drive/__TECH/_My Flask Apps/Disney Picture Material"

#Put files into lists and return them as one list with all images in the folder
def loadImages(path):
    image_file = sorted([os.path.join(path, file)
                          for file in os.listdir(path )
                          if file.endswith('.JPG')])
    return image_file

#Define image_list & Convert to numpy array
image_list = loadImages(image_path)
path = np.array(image_list)
path_string = (path[1])
img = tf.io.read_file(path_string)
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32)
#print(img)

IMG_HEIGHT = 150
IMG_WIDTH = 150
final_img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

#Print Image + Details
plt.subplot(121), plt.imshow(final_img)
print(final_img.shape)
final_img_tfl = np.expand_dims(final_img, axis=0)
print(final_img_tfl.shape)

In [ ]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

## Data preparation

In [ ]:
train_image_generator = ImageDataGenerator( rescale=1./255,
                                            rotation_range=40,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode='nearest') # Generator for our training data

validation_image_generator = ImageDataGenerator(rescale=1./255,
                                                rotation_range=40,
                                                width_shift_range=0.2,
                                                height_shift_range=0.2,
                                                shear_range=0.2,
                                                zoom_range=0.2,
                                                horizontal_flip=True,
                                                fill_mode='nearest') # Generator for our validation data

After defining the generators for training and validation images, the `flow_from_directory` method load images from the disk, applies rescaling, and resizes the images into the required dimensions.

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=5,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical'
                                                         )

print(train_data_gen)
print(type(train_data_gen))

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=5,
                                                             directory=validation_dir,
                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                            class_mode='categorical')

In [ ]:
labels = (train_data_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

### Visualize training images

Visualize the training images by extracting a batch of images from the training generator—which is 32 images in this example—then plot five of them with `matplotlib`.

In [ ]:
sample_training_images, _ = next(train_data_gen)

The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is training features and y_train, its labels. Discard the labels to only visualize the training images.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

## Create the model

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 512 units on top of it that is activated by a `relu` activation function.

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
#IMG_HEIGHT = 150
#IMG_WIDTH = 150
final_img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

final_img_tfl = np.expand_dims(final_img, axis=0)
print(final_img_tfl.shape)

feature_batch = base_model(final_img_tfl)
print(feature_batch.shape)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(units = 14, input_shape = (520,), activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

Compile the model

For this tutorial, choose the *ADAM* optimizer and *binary cross entropy* loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Model summary

View all the layers of the network using the model's `summary` method:

In [ ]:
model.summary()

### Train the model

Use the `fit_generator` method of the `ImageDataGenerator` class to train the network.

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=5,
    epochs=100,
    verbose=1,
    validation_data=val_data_gen,
    validation_steps=2
)

### Visualize training results

Now visualize the results after training the network.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save('disney_model_2.h5')

In [ ]:
image_path = "/content/gdrive/My Drive/Datasets"

def loadImages(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_file = sorted([os.path.join(path, file)
                          for file in os.listdir(path )
                          if file.endswith('.png')])
    return image_file

image_list = loadImages(image_path)
print(image_list)

path = np.array(image_list)
path_string = (path[0])

print(path_string)

img = tf.io.read_file(path_string)
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32)
final_img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])


plt.subplot(121), plt.imshow(final_img)

In [ ]:
#Expand Tensor for Model (Input shape)
y = np.expand_dims(final_img, axis=0)

#Predict Image Tensor with model
prediction = model.predict(y)
prediction_squeeze = np.squeeze(prediction, axis=0)

label_array = np.array(labels)

#print(type(label))
for key, value in labels.items():
    real_label = prediction_squeeze[key]
    
    print ("{0:.0%}".format(real_label), value)

In [ ]:
#predictions = [labels[k] for k in predicted_class_indices]